# Import Library

In [1]:
import gmaps
import pandas as pd
import numpy as np
import requests
import json
from config import gkey
# Configure gmaps
gmaps.configure(api_key=gkey)

# Read csv file

In [2]:
# Create a dataframe
weather_df = pd.read_csv('../WeatherPy/weatherpy_data.csv')
weather_df.head()

,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,ericeira,pt,2020-03-14 12:49:18,38.96,-9.42,71.60,63.0,0.0,12.75
1,souillac,mu,2020-03-14 12:51:56,-20.52,57.52,78.80,83.0,40.0,3.36
2,bengkulu,id,2020-03-14 12:51:56,-3.80,102.27,75.18,90.0,100.0,3.80
3,liverpool,ca,2020-03-14 12:50:48,44.03,-64.72,46.99,51.0,25.0,13.00
4,ileza,ru,2020-03-14 12:51:57,61.05,43.90,13.62,88.0,72.0,5.14


## Create a heat map that displays the humidity for every city 

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]
Humidity = weather_df["Humidity (%)"].astype(float)
maxhumidity = Humidity.max()

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity,
                                 dissipating=False, max_intensity=maxhumidity,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Find ideal weather condition

### Narrow down the DataFrame to find an ideal weather condition.

* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.

In [5]:
new_weather_df = weather_df.loc[(weather_df["Max Temperature (F)"] > 70) & 
                                (weather_df["Max Temperature (F)"] < 80) & 
                                (weather_df["Wind Speed (mph)"]<10) &
                                (weather_df["Cloudiness (%)"] == 0) , :]

new_weather_df.reset_index(inplace=True)
del new_weather_df['index']
print(f"There are {len(new_weather_df)} cities satisfying the ideal weather condition.")
new_weather_df

There are 8 cities satisfying the ideal weather condition.


,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,matara,lk,2020-03-14 12:52:03,5.95,80.54,74.82,83.0,0.0,5.73
1,srivardhan,in,2020-03-14 12:52:26,18.03,73.02,71.94,58.0,0.0,5.75
2,kununurra,au,2020-03-14 12:52:32,-15.77,128.73,77.00,94.0,0.0,4.70
3,langsa,id,2020-03-14 12:52:41,4.47,97.97,76.98,88.0,0.0,2.77
4,huejuquilla el alto,mx,2020-03-14 12:52:48,22.60,-103.87,79.32,20.0,0.0,2.62
5,adrar,dz,2020-03-14 12:53:01,25.75,-1.00,75.90,10.0,0.0,2.82
6,mundgod,in,2020-03-14 12:55:49,14.97,75.03,72.48,49.0,0.0,7.11
7,bilma,ne,2020-03-14 12:54:39,18.69,12.92,71.58,13.0,0.0,7.29


## Build a Hotel Map

In [6]:
hotels = []

for i in range(len(new_weather_df)):
    lat = new_weather_df.loc[i]['Latitude']
    lng = new_weather_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": gkey
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    try:
        print(f"Processing Record {i}, City Name: {new_weather_df.loc[i]['City']}.")
        hotels.append(response['results'][0]['name'])
        
    except:
        hotels.append(np.nan)
        print("Oops, Nearest hotel not found!")
        
print("-----------------------------")        
print("Data Retrieval Complete")
print("-----------------------------")

Processing Record 0, City Name: matara.
Processing Record 1, City Name: srivardhan.
Processing Record 2, City Name: kununurra.
Processing Record 3, City Name: langsa.
Processing Record 4, City Name: huejuquilla el alto.
Processing Record 5, City Name: adrar.
Processing Record 6, City Name: mundgod.
Processing Record 7, City Name: bilma.
-----------------------------
Data Retrieval Complete
-----------------------------


In [7]:
weather_data=new_weather_df.copy()
weather_data["Hotel Name"] = hotels
weather_data=weather_data.dropna()
weather_data

,City,Country,Date,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,matara,lk,2020-03-14 12:52:03,5.95,80.54,74.82,83.0,0.0,5.73,Matara
1,srivardhan,in,2020-03-14 12:52:26,18.03,73.02,71.94,58.0,0.0,5.75,Shrivardhan
2,kununurra,au,2020-03-14 12:52:32,-15.77,128.73,77.00,94.0,0.0,4.70,Kununurra
3,langsa,id,2020-03-14 12:52:41,4.47,97.97,76.98,88.0,0.0,2.77,Langsa
4,huejuquilla el alto,mx,2020-03-14 12:52:48,22.60,-103.87,79.32,20.0,0.0,2.62,Huejuquilla El Alto
5,adrar,dz,2020-03-14 12:53:01,25.75,-1.00,75.90,10.0,0.0,2.82,Reggane
6,mundgod,in,2020-03-14 12:55:49,14.97,75.03,72.48,49.0,0.0,7.11,Mundgod
7,bilma,ne,2020-03-14 12:54:39,18.69,12.92,71.58,13.0,0.0,7.29,Bilma


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in weather_data.iterrows()]
locations = weather_data[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))